# Let's get SQLical!

In this project, we use SQL to analyze some data from the CIA Factbook. The CIA Factbook is a compendium of countries and some of their attributes, for example: population, land area, birth rate, death rate, migration rate, etc. We use this dataset to answer a couple of questions, including the following:
- What is the minimum and maximum population and population growth?
- What countries are most densely populated?
- Which countries have the highest ratio of water to land?
- What countries are at risk of overpopulation? What about underpopulation?

This dataset catalogs contries according to some attributes, including:

|Column name|Description|
|---|---|
|code|Two letter code used to abbreviate country names|
|name|The names of the countries in question|
|area|The area of the country (not sure what unit it's in)|
|area_land|The area of the country comprised of land|
|area_water|The area of the country comprised of water|
|population|The population of the country|
|population_growth|The country's annual population growth rate|
|birth_rate|number of births per 1000 people in the population, annually|
|death_rate|number of deaths per 1000 people in the population, annually|
|migration_rate|number of migrations (net emigrations vs immigrations) per 1000 people|

This project is largely exploratory, without a specific objective.

## 1. Data Overview

First let's load SQL into our notebook

In [129]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Let's now examine the contents of factbook.db (the database we'll be using in this project).

In [130]:
%%sql
SELECT *
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Looks like there are two tables within this database. We'll be working only with the `facts` table in this project. Let's display the first five rows of the `facts` table, to ensure we can access it properly.

In [131]:
%%sql
SELECT *
    FROM facts
    LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## 2. Data Analysis
In the following sections we'll be digging into the data by coming up with (and answering) some questions about the data

### 2.1 - What is the minimum and maximum population and population growth in the data?

In [132]:
%%sql
SELECT MIN("population"),MAX("population"),MIN("population_growth"),MAX("population_growth")
    FROM facts

 * sqlite:///factbook.db
Done.


"MIN(""population"")","MAX(""population"")","MIN(""population_growth"")","MAX(""population_growth"")"
0,7256490011,0.0,4.02


Looks like the maximum population of a country is about 7.2 billion people (which is about the population of the whole world). Something about this entry seems off, so let's take a deeper look later.

#### 2.1.1 - What country has the minimum population?

In [133]:
%%sql
SELECT "name","population"
    FROM facts
    WHERE "population" = (SELECT MIN("population")
                              FROM facts);

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0


#### 2.1.2 - What country has the maximum population?
Earlier we saw a country with ~7.2 billion people as its population. Let's take a deeper look by first identifying this country.

In [134]:
%%sql
SELECT "name","population"
    FROM facts
    WHERE "population" = (SELECT MAX("population")
                              FROM facts);

 * sqlite:///factbook.db
Done.


name,population
World,7256490011


Looks like our `facts` table has an entry for the entire world, and that's the entry that is being problematic. We'll need to re-examine the question of "What is the highest population in a country" while excluding this entry, but first let's look at the entire entry for "World"

In [135]:
%%sql
SELECT *
    FROM facts
    WHERE "name" = 'World'

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Now let's re-examine the highest population question.

####

In [136]:
%%sql
SELECT MIN("population"),MAX("population"),MIN("population_growth"),MAX("population_growth")
    FROM facts
    WHERE name != "World"

 * sqlite:///factbook.db
Done.


"MIN(""population"")","MAX(""population"")","MIN(""population_growth"")","MAX(""population_growth"")"
0,1367485388,0.0,4.02


In [137]:
%%sql
SELECT "name","population"
    FROM facts
    WHERE "population" = (SELECT MAX("population")
                              FROM facts
                              WHERE "name" != "World")

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


That looks a lot better. China has the maximum population, at about 1.3 billion people.

### 2.2 - What is the average population and average area of countries in the data?

In [138]:
%%sql
SELECT AVG("population"), AVG("area")
    FROM facts
    WHERE "name" != "World"

 * sqlite:///factbook.db
Done.


"AVG(""population"")","AVG(""area"")"
32242666.56846473,555093.546184739


### 2.3 - What countries are most densely populated?

#### 2.3.1 - Define density (Part A)
Let's find the most densely populated countries, defined as countries with:
* Below average area and
* Above average population

In [139]:
%%sql
SELECT "name","population","area"
    FROM facts
    WHERE ("population" > (SELECT AVG("population") FROM facts
                          WHERE "name" != "World"))
    AND ("area" < (SELECT AVG("area") FROM facts))

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


#### 2.3.2 - Define density (Part B)
Now let's find the most densely populated countries with density defined as the population divided by area.

In [140]:
%%sql
SELECT "name",(CAST("population" AS FLOAT)/CAST("area" AS FLOAT)) AS "density"
FROM facts
WHERE ("area" != "None")
AND ("population" != "None")
ORDER BY "density" DESC
LIMIT 10

 * sqlite:///factbook.db
Done.


name,density
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


Quite a different result. It looks like the most densely populated country here is Macau. I'm inclined to go with this definition of density, as our previoius definition assumes that:
1. Countries with lower than average population have low densities as defined here
2. Countries with larger than average area have low densities as defined here.

### 2.4 - What country has the highest growth rate?

In [141]:
%%sql
SELECT "name","population_growth"
    FROM facts
    WHERE "population_growth" = (SELECT MAX("population_growth")
                               FROM facts)

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


Looks like South Sudan has the highest growth rate, with 4.02 people for every 1000.

### 2.5 - Which countries have the highest ratios of water to land?

In [142]:
%%sql
SELECT "name",(CAST("area_water" AS FLOAT)/CAST("area_land" AS FLOAT)) AS "water_land_ratio"
    FROM facts
    ORDER BY "water_land_ratio" DESC
    LIMIT 5

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623


The countries with the highest ratios of water to land (i.e countries which have a significant part of their area covered by water), are:
1. British Indian Ocean Territory
2. Virgin Islands
3. Puerto Rico
4. The Bahamas
5. Guinea-Bissau

A lot of these nations are surrounded by water, so this makes sense.

### 2.6 - Which countries have more water than land?

In [143]:
%%sql
SELECT "name",(CAST("area_water" AS FLOAT)/CAST("area_land" AS FLOAT)) AS "water_land_ratio"
    FROM facts
    WHERE "water_land_ratio" > 1
    ORDER BY "water_land_ratio" DESC
    LIMIT 5

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832


The British Indian Ocean Territory as well as the Virgin Islands are the only two nations in this database (which I believe is quite exhaustive) that have more water than land area. Makes sense that the nations would be majorly comprised of island nations.

### 2.7 - Which countries will add the most people to their populations next year?

In other words, which countries have the highest `population_growth`? We already know the country with the highest population growth rate (South Sudan). Let's find the top five.

In [144]:
%%sql
SELECT "name","population_growth"
    FROM facts
    ORDER BY "population_growth" DESC
    LIMIT 5

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


The countries with the top five population growth rates are (in descending order):
1. South Sudan
2. Malawi
3. Burundi
4. Niger
5. Uganda

It is interesting that the countries with the highest growth rates are concentrated in Africa. This could be something interesting to dig into...good news is at least these countries are not among the top 10 most densely populated in the World. Perhaps it's worth investigating if they're even in the top 50% of dense countries.

#### 2.7.1 - Countries with risk of overpopulation

Let's answer the question: of the top 5 countries with highest `population_growth`, which of them already in the top 50% of countries for population density?

In [145]:
%%sql
CREATE VIEW top_n_density AS
    SELECT "name"
    FROM (SELECT "name", (CAST("population" AS FLOAT))/(CAST("area" AS FLOAT)) AS "density"
          FROM facts
          WHERE ("population" != "None")
          AND ("area" != "None")
          ORDER BY "density" DESC
          LIMIT (SELECT COUNT(*)/2 FROM facts
                WHERE "name" != "World")
         )
;
    
CREATE VIEW top_5_pop_growth AS
    SELECT "name"
    FROM facts
    ORDER BY "population_growth" DESC
    LIMIT 5
;

SELECT *
FROM (
    SELECT * FROM top_n_density INTERSECT SELECT * FROM top_5_pop_growth
)

 * sqlite:///factbook.db
Done.
Done.
Done.


name
Burundi
Malawi
Uganda


In [146]:
%%sql
DROP VIEW top_n_density;
DROP VIEW top_5_pop_growth

 * sqlite:///factbook.db
Done.
Done.


[]

Looks like three of the top five highest population growth countries are among the top 50% most densely populated countries. And all three countries are in Africa. This might be the tip of the spear of a pattern. A next step could be to investigate whether African nations are disproportionately at risk for overpopulation compared to the rest of the world.

### 2.8 - Which countries are at highest risk of underpopulation?

Let's define countries at risk of underpopulation as countries with a higher death rate than birth rate.

In [147]:
%%sql
SELECT "name","birth_rate","death_rate"
    FROM facts
    WHERE "death_rate" > "birth_rate"
    ORDER BY ("death_rate" - "birth_rate") DESC

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Bulgaria,8.92,14.44
Serbia,9.08,13.66
Latvia,10.0,14.31
Lithuania,10.1,14.27
Ukraine,10.72,14.46
Hungary,9.16,12.73
Germany,8.47,11.42
Slovenia,8.42,11.37
Romania,9.14,11.9
Croatia,9.45,12.18


In [148]:
%%sql
SELECT COUNT(*)
    FROM facts
    WHERE "death_rate" > "birth_rate"
    ORDER BY ("death_rate" - "birth_rate") DESC

 * sqlite:///factbook.db
Done.


COUNT(*)
24


The 24 countries above have a death rate greater than their birth rate, and per our definition, are at a greater risk for underpopulation.

In this project, we've used SQL to explore a dataset cataloging countries in their world by some select attributes. We answered a good couple questions, including
* Which countries are at highest risk of overpopulation
* Which countries are at highest risk of underpopulation

A possible next step is to merge this dataset with a dataset that catalogs countries by their continent, and investigating if there is a trend of overpopulation in African countries, as we saw with three African countries above.